In [27]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import packages and libraries
#setting up initialization

In [28]:
#Read in the csv file from kaggle that contains the San Francisco Building Permits dataset
dfpermits = pd.read_csv('Building_Permits.csv')

C:\Users\taqua\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
#Dropping columns that I feel are somewhat unneccessary to my project, have too much missing, 
#or I couldn't understand what the column was for 
dfpermits = dfpermits.drop(['Street Number Suffix', 'Street Suffix', 'Unit', 'Unit Suffix', 
                            'TIDF Compliance', 'Current Status Date', 'Completed Date', 
                            'First Construction Document Date', 'Structural Notification',
                            'Voluntary Soft-Story Retrofit', 'Fire Only Permit', 'Permit Expiration Date',
                            'Site Permit', 'Street Number', 'Street Name', 'Existing Use',
                            'Record ID', 'Existing Units', 'Plansets', 'Proposed Use', 'Proposed Units'], axis = 1)

In [30]:
#Check what columns I have left
dfpermits.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition',
       'Permit Creation Date', 'Block', 'Lot', 'Description', 'Current Status',
       'Filed Date', 'Issued Date', 'Number of Existing Stories',
       'Number of Proposed Stories', 'Estimated Cost', 'Revised Cost',
       'Existing Construction Type', 'Existing Construction Type Description',
       'Proposed Construction Type', 'Proposed Construction Type Description',
       'Supervisor District', 'Neighborhoods - Analysis Boundaries', 'Zipcode',
       'Location'],
      dtype='object')

In [31]:
#drop rows of more than 15 missing values
dfpermits.dropna(thresh=22, inplace =True)

#return new shape of the dataframe
dfpermits.shape

(138649, 22)

In [32]:
dfpermits.isnull().sum()

Permit Number                             0
Permit Type                               0
Permit Type Definition                    0
Permit Creation Date                      0
Block                                     0
Lot                                       0
Description                               0
Current Status                            0
Filed Date                                0
Issued Date                               0
Number of Existing Stories                0
Number of Proposed Stories                0
Estimated Cost                            0
Revised Cost                              0
Existing Construction Type                0
Existing Construction Type Description    0
Proposed Construction Type                0
Proposed Construction Type Description    0
Supervisor District                       0
Neighborhoods - Analysis Boundaries       0
Zipcode                                   0
Location                                  0
dtype: int64

In [33]:
dfpermits.shape

(138649, 22)

In [34]:
dfpermits['Filed Date'] = pd.to_datetime(dfpermits['Filed Date'])
dfpermits['Issued Date'] = pd.to_datetime(dfpermits['Issued Date'])

In [35]:
#New attribute necessary for my prediction
dfpermits['Time'] = (dfpermits['Issued Date'] - dfpermits['Filed Date']) / np.timedelta64(1, 'D')

In [36]:
dfpermits.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Description,Current Status,Filed Date,Issued Date,...,Revised Cost,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Time
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,repair dryrot & stucco at front of bldg.,complete,2016-11-07,2017-07-18,...,2000.0,5.0,wood frame (5),5.0,wood frame (5),3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",253.0
5,201706149344,8,otc alterations permit,06/14/2017,4105,009,evac maps,issued,2017-06-14,2017-07-06,...,4000.0,1.0,constr type 1,1.0,constr type 1,10.0,Potrero Hill,94107.0,"(37.75922331346539, -122.39170402628598)",22.0
6,201706300814,8,otc alterations permit,06/30/2017,1739,020,re-roofing,complete,2017-06-30,2017-06-30,...,12000.0,5.0,wood frame (5),5.0,wood frame (5),5.0,Inner Sunset,94122.0,"(37.764145640138565, -122.46875112470363)",0.0
24,201708084249,8,otc alterations permit,08/08/2017,0542,039,re-roofing,issued,2017-08-08,2017-08-08,...,64650.0,5.0,wood frame (5),5.0,wood frame (5),2.0,Marina,94123.0,"(37.79739093461306, -122.43127133823367)",0.0
25,201708165014,8,otc alterations permit,08/16/2017,1824,006,re-roofing,issued,2017-08-16,2017-08-16,...,7000.0,5.0,wood frame (5),5.0,wood frame (5),4.0,Sunset/Parkside,94122.0,"(37.76052279949518, -122.4879889739165)",0.0


In [37]:
dfpermits.isnull().sum()

Permit Number                             0
Permit Type                               0
Permit Type Definition                    0
Permit Creation Date                      0
Block                                     0
Lot                                       0
Description                               0
Current Status                            0
Filed Date                                0
Issued Date                               0
Number of Existing Stories                0
Number of Proposed Stories                0
Estimated Cost                            0
Revised Cost                              0
Existing Construction Type                0
Existing Construction Type Description    0
Proposed Construction Type                0
Proposed Construction Type Description    0
Supervisor District                       0
Neighborhoods - Analysis Boundaries       0
Zipcode                                   0
Location                                  0
Time                            

In [38]:
dfpermits['cat_time'] = pd.cut(dfpermits.Time, bins = [0,1,7,15,31,62,93,124,155,186,365,1800]
                                      , labels =['Same day', 'Less than a week' ,'1-2 weeks','2 weeks - 1 month', 
                                                 '1-2 months', '2-3 months', '3-4 months', '4-5 months', 
                                                 '5-6 months', '6 months-a year', '1+ year']
                                      , right=False)

dfpermits.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Description,Current Status,Filed Date,Issued Date,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Time,cat_time
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,repair dryrot & stucco at front of bldg.,complete,2016-11-07,2017-07-18,...,5.0,wood frame (5),5.0,wood frame (5),3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",253.0,6 months-a year
5,201706149344,8,otc alterations permit,06/14/2017,4105,009,evac maps,issued,2017-06-14,2017-07-06,...,1.0,constr type 1,1.0,constr type 1,10.0,Potrero Hill,94107.0,"(37.75922331346539, -122.39170402628598)",22.0,2 weeks - 1 month
6,201706300814,8,otc alterations permit,06/30/2017,1739,020,re-roofing,complete,2017-06-30,2017-06-30,...,5.0,wood frame (5),5.0,wood frame (5),5.0,Inner Sunset,94122.0,"(37.764145640138565, -122.46875112470363)",0.0,Same day
24,201708084249,8,otc alterations permit,08/08/2017,0542,039,re-roofing,issued,2017-08-08,2017-08-08,...,5.0,wood frame (5),5.0,wood frame (5),2.0,Marina,94123.0,"(37.79739093461306, -122.43127133823367)",0.0,Same day
25,201708165014,8,otc alterations permit,08/16/2017,1824,006,re-roofing,issued,2017-08-16,2017-08-16,...,5.0,wood frame (5),5.0,wood frame (5),4.0,Sunset/Parkside,94122.0,"(37.76052279949518, -122.4879889739165)",0.0,Same day


In [39]:
dfpermits.shape

(138649, 24)

In [40]:
dfpermits['cat_time'].isnull().sum()

0

In [41]:
dfpermits['Permit Type Definition'].value_counts()

otc alterations permit              129861
additions alterations or repairs      8787
new construction wood frame              1
Name: Permit Type Definition, dtype: int64

In [42]:
#drop columns with categorical values
dfpermits.drop(columns=['Permit Number', 'Permit Type Definition', 
                        'Permit Creation Date', "Block", 'Description', "Lot", "Current Status", 
                        "Filed Date", "Issued Date",
                        "Existing Construction Type Description", "Proposed Construction Type Description", 
                        "Neighborhoods - Analysis Boundaries", "Location"], inplace = True)

In [43]:
dfpermits

,Permit Type,Number of Existing Stories,Number of Proposed Stories,Estimated Cost,Revised Cost,Existing Construction Type,Proposed Construction Type,Supervisor District,Zipcode,Time,cat_time
3,8,2.0,2.0,2000.00,2000.00,5.0,5.0,3.0,94109.0,253.0,6 months-a year
5,8,5.0,5.0,4000.00,4000.00,1.0,1.0,10.0,94107.0,22.0,2 weeks - 1 month
6,8,3.0,3.0,12000.00,12000.00,5.0,5.0,5.0,94122.0,0.0,Same day
24,8,4.0,4.0,64650.00,64650.00,5.0,5.0,2.0,94123.0,0.0,Same day
25,8,2.0,2.0,7000.00,7000.00,5.0,5.0,4.0,94122.0,0.0,Same day
...,...,...,...,...,...,...,...,...,...,...,...
198873,8,2.0,2.0,1.00,1.00,5.0,5.0,9.0,94110.0,0.0,Same day
198874,8,3.0,3.0,10000.00,15000.00,5.0,5.0,8.0,94131.0,0.0,Same day
198877,8,2.0,2.0,1000.00,1000.00,5.0,5.0,11.0,94112.0,0.0,Same day
198879,8,2.0,2.0,18453.12,18453.12,5.0,5.0,11.0,94112.0,0.0,Same day


In [44]:
dfpermits.isnull().sum()

Permit Type                   0
Number of Existing Stories    0
Number of Proposed Stories    0
Estimated Cost                0
Revised Cost                  0
Existing Construction Type    0
Proposed Construction Type    0
Supervisor District           0
Zipcode                       0
Time                          0
cat_time                      0
dtype: int64

In [45]:
dfpermits.dtypes

Permit Type                      int64
Number of Existing Stories     float64
Number of Proposed Stories     float64
Estimated Cost                 float64
Revised Cost                   float64
Existing Construction Type     float64
Proposed Construction Type     float64
Supervisor District            float64
Zipcode                        float64
Time                           float64
cat_time                      category
dtype: object

In [46]:
dfpermits

,Permit Type,Number of Existing Stories,Number of Proposed Stories,Estimated Cost,Revised Cost,Existing Construction Type,Proposed Construction Type,Supervisor District,Zipcode,Time,cat_time
3,8,2.0,2.0,2000.00,2000.00,5.0,5.0,3.0,94109.0,253.0,6 months-a year
5,8,5.0,5.0,4000.00,4000.00,1.0,1.0,10.0,94107.0,22.0,2 weeks - 1 month
6,8,3.0,3.0,12000.00,12000.00,5.0,5.0,5.0,94122.0,0.0,Same day
24,8,4.0,4.0,64650.00,64650.00,5.0,5.0,2.0,94123.0,0.0,Same day
25,8,2.0,2.0,7000.00,7000.00,5.0,5.0,4.0,94122.0,0.0,Same day
...,...,...,...,...,...,...,...,...,...,...,...
198873,8,2.0,2.0,1.00,1.00,5.0,5.0,9.0,94110.0,0.0,Same day
198874,8,3.0,3.0,10000.00,15000.00,5.0,5.0,8.0,94131.0,0.0,Same day
198877,8,2.0,2.0,1000.00,1000.00,5.0,5.0,11.0,94112.0,0.0,Same day
198879,8,2.0,2.0,18453.12,18453.12,5.0,5.0,11.0,94112.0,0.0,Same day


In [47]:
X = dfpermits.iloc[:, 0:9]
Y = dfpermits.iloc[:, -1]

In [48]:
X.values

array([[8.0000e+00, 2.0000e+00, 2.0000e+00, ..., 5.0000e+00, 3.0000e+00,
        9.4109e+04],
       [8.0000e+00, 5.0000e+00, 5.0000e+00, ..., 1.0000e+00, 1.0000e+01,
        9.4107e+04],
       [8.0000e+00, 3.0000e+00, 3.0000e+00, ..., 5.0000e+00, 5.0000e+00,
        9.4122e+04],
       ...,
       [8.0000e+00, 2.0000e+00, 2.0000e+00, ..., 5.0000e+00, 1.1000e+01,
        9.4112e+04],
       [8.0000e+00, 2.0000e+00, 2.0000e+00, ..., 5.0000e+00, 1.1000e+01,
        9.4112e+04],
       [8.0000e+00, 2.0000e+00, 2.0000e+00, ..., 5.0000e+00, 1.1000e+01,
        9.4112e+04]])

In [49]:
Y.values

[6 months-a year, 2 weeks - 1 month, Same day, Same day, Same day, ..., Same day, Same day, Same day, Same day, Same day]
Length: 138649
Categories (11, object): [Same day < Less than a week < 1-2 weeks < 2 weeks - 1 month ... 4-5 months < 5-6 months < 6 months-a year < 1+ year]

In [50]:
# Bagged Decision Trees for Classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
filename = 'Building_Permits.csv'
names = ['Permit Type', 'Street Number',
       'Number of Existing Stories', 'Number of Proposed Stories',
       'Estimated Cost', 'Revised Cost', 'Existing Units', 'Proposed Units',
       'Existing Construction Type', 'Proposed Construction Type',
       'Supervisor District', 'Zipcode', 'Time']

In [51]:
dataframe = read_csv(filename, names=names)
array = dataframe.values
seed = 7
kfold = KFold(n_splits=10, random_state=seed)
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

C:\Users\taqua\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1,6,10,20,21,22,25,26,28,30,31,32,33,35,38,40,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\taqua\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.5353446904440207


In [52]:
# Random Forest Classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
filename = 'Building_Permits.csv'
names = ['Permit Type', 'Street Number',
       'Number of Existing Stories', 'Number of Proposed Stories',
       'Estimated Cost', 'Revised Cost', 'Existing Units', 'Proposed Units',
       'Existing Construction Type', 'Proposed Construction Type',
       'Supervisor District', 'Zipcode', 'Time']
dataframe = read_csv(filename, names=names)
array = dataframe.values
num_trees = 100
max_features = 3
kfold = KFold(n_splits=10, random_state=7)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

C:\Users\taqua\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1,6,10,20,21,22,25,26,28,30,31,32,33,35,38,40,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\taqua\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.5383378662308981
